In [1]:
import cPickle as pkl
import pandas as pd
import numpy as np
import os.path

In [2]:
dirname = '../../results_20180430_2/'

In [3]:
files = !ls {dirname}
files = [os.path.join(dirname, filename) for filename in files]
files

['../../results_20180430_2/test configuration with 2 genes, transcripts 2000, 2000, init rates 0.1, 0.001, 100 ribosomes, deltat=0.1s_20180430_1510_100_ribosomes_0200s.p',
 '../../results_20180430_2/test configuration with 2 genes, transcripts 2000, 2000, init rates 0.1, 0.001, 10 ribosomes, deltat=0.1s_20180430_1508_10_ribosomes_0200s.p',
 '../../results_20180430_2/test configuration with 2 genes, transcripts 2000, 2000, init rates 0.1, 0.001, 1 ribosomes, deltat=0.1s_20180430_1506_1_ribosomes_0200s.p',
 '../../results_20180430_2/test configuration with 2 genes, transcripts 2000, 2000, init rates 0.1, 0.001, 200 ribosomes, deltat=0.1s_20180430_1510_200_ribosomes_0200s.p',
 '../../results_20180430_2/test configuration with 2 genes, transcripts 2000, 2000, init rates 0.1, 0.001, 20 ribosomes, deltat=0.1s_20180430_1508_20_ribosomes_0200s.p',
 '../../results_20180430_2/test configuration with 2 genes, transcripts 2000, 2000, init rates 0.1, 0.001, 2 ribosomes, deltat=0.1s_20180430_1507_2_

In [4]:
descriptor = files[0].split('/')[-1][:-2]
descriptor

'test configuration with 2 genes, transcripts 2000, 2000, init rates 0.1, 0.001, 100 ribosomes, deltat=0.1s_20180430_1510_100_ribosomes_0200s'

Blacklisted species:

In [5]:
blacklist = ['ribos._bound', 'ribos._free', 'tRNA_free', 'tRNA_bound', 'ATP', 'GTP', 'AMP', 'GDP', 'protein', 
             'proteins', 'peptide_bonds']

Parameters for efficiency:

In [6]:
sim_time = 200.
timepoints = sim_time / 0.1
print timepoints

2000.0


In [7]:
post_burn_in = 100.  # seconds
share_of_total = post_burn_in / sim_time
print share_of_total

0.5


In [8]:
points_for_efficiency = int(share_of_total * timepoints + 1)
print points_for_efficiency

1001


Data frame at cell level:

In [9]:
dfc = pd.DataFrame(columns=['ribosomes', 'transcripts', 'collisions', 'nocollisions', 'peptide_bonds'])

In [10]:
rows = []
dft_columns = ['length', 'init_rate', 'ribosomes', 'tic_toc', 'geneID', 'transcripts', 'index']

for i, filename in enumerate(files):
    print filename
    results = pkl.load(open(filename))
    
    # append to cell level data frame
    print 'cell level...'
    rowvals = [results['n_ribosomes'], len(results['transcriptome']), results['collisions'], 
               results['nocollisions'], results['peptide_bonds']]
    row = dict(zip(list(dfc.columns), rowvals))
    #print row
    dfc = dfc.append(row, ignore_index=True)
    #print dfc.head()
    
    # append to transcript level data frame
    print 'transcript level...'
    
    for transcript in results['transcriptome']:
        av_time = np.mean([tictoc[1] - tictoc[0] for tictoc in transcript.tic_toc])
        rowvals = [transcript.geneID, transcript.index, transcript.init_rate, transcript.length, 
                   results['n_ribosomes'], av_time, len(results['transcriptome'])]
        row = dict(zip(list(dft_columns), rowvals))
        #print row
        rows.append(row)
            
    # append to gene level data frame
    print 'gene level...'
    header = str(results['n_ribosomes']) + '_' + str(len(results['transcriptome']))
    tc = results['timecourses']
    efficiency_items = [(gene, (tc[gene][-1] - tc[gene][-points_for_efficiency]) / post_burn_in) for gene in tc 
                        if gene not in blacklist]
    tmp = pd.DataFrame(efficiency_items, columns = ['gene', header]).set_index('gene')
    
    if i == 0:  # first time
        dfg = tmp
    else:
        dfg = pd.merge(dfg, tmp, left_index=True, right_index=True, how='outer')

dfc.sort_values('ribosomes', inplace=True)

dft = pd.DataFrame(rows)
dft.columns = ['ribosomes', 'transcripts', 'index', 'geneID', 'init_rate', 'length', 'tic_toc']

../../results_20180430_2/test configuration with 2 genes, transcripts 2000, 2000, init rates 0.1, 0.001, 100 ribosomes, deltat=0.1s_20180430_1510_100_ribosomes_0200s.p
cell level...
transcript level...
gene level...
../../results_20180430_2/test configuration with 2 genes, transcripts 2000, 2000, init rates 0.1, 0.001, 10 ribosomes, deltat=0.1s_20180430_1508_10_ribosomes_0200s.p


/home/martin/.local/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/martin/.local/lib/python2.7/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


cell level...
transcript level...
gene level...
../../results_20180430_2/test configuration with 2 genes, transcripts 2000, 2000, init rates 0.1, 0.001, 1 ribosomes, deltat=0.1s_20180430_1506_1_ribosomes_0200s.p
cell level...
transcript level...
gene level...
../../results_20180430_2/test configuration with 2 genes, transcripts 2000, 2000, init rates 0.1, 0.001, 200 ribosomes, deltat=0.1s_20180430_1510_200_ribosomes_0200s.p
cell level...
transcript level...
gene level...
../../results_20180430_2/test configuration with 2 genes, transcripts 2000, 2000, init rates 0.1, 0.001, 20 ribosomes, deltat=0.1s_20180430_1508_20_ribosomes_0200s.p
cell level...
transcript level...
gene level...
../../results_20180430_2/test configuration with 2 genes, transcripts 2000, 2000, init rates 0.1, 0.001, 2 ribosomes, deltat=0.1s_20180430_1507_2_ribosomes_0200s.p
cell level...
transcript level...
gene level...
../../results_20180430_2/test configuration with 2 genes, transcripts 2000, 2000, init rates 0.1, 

In [11]:
dfc

,ribosomes,transcripts,collisions,nocollisions,peptide_bonds
2,1.0,6000.0,0.0,104.0,1090.0
5,2.0,6000.0,0.0,205.0,2155.0
7,3.0,6000.0,0.0,310.0,3134.0
10,5.0,6000.0,0.0,501.0,5270.0
13,8.0,6000.0,1.0,796.0,8425.0
1,10.0,6000.0,3.0,990.0,10469.0
4,20.0,6000.0,21.0,1997.0,21050.0
6,30.0,6000.0,45.0,2935.0,31335.0
9,50.0,6000.0,114.0,4917.0,52499.0
12,80.0,6000.0,282.0,7956.0,84821.0


In [12]:
dft = dft.dropna(subset=['tic_toc'])
dft

,ribosomes,transcripts,index,geneID,init_rate,length,tic_toc
0,100,6000,0,0,0.10,30,1.388889
1,100,6000,1,0,0.10,30,1.300000
2,100,6000,2,0,0.10,30,1.480000
3,100,6000,3,0,0.10,30,1.575000
4,100,6000,4,0,0.10,30,1.800000
5,100,6000,5,0,0.10,30,1.425000
6,100,6000,6,0,0.10,30,1.450000
7,100,6000,7,0,0.10,30,1.528571
8,100,6000,8,0,0.10,30,0.933333
9,100,6000,9,0,0.10,30,1.833333


In [13]:
dfg

,100_6000,10_6000,1_6000,200_6000,20_6000,2_6000,30_6000,3_6000,500_6000,50_6000,5_6000,800_6000,80_6000,8_6000
gene,,,,,,,,,,,,,,
0,42.56,4.25,0.40,83.66,8.67,0.93,11.99,1.33,206.43,21.36,2.22,317.74,33.64,3.57
1,0.48,0.04,0.01,0.85,0.07,0.00,0.13,NaN,2.41,0.23,0.00,4.15,0.32,0.01
2,4.39,0.41,0.05,9.32,0.87,0.08,1.40,0.11,23.44,2.19,0.21,39.87,3.71,0.33
tRNA_free_01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
tRNA_free_02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
tRNA_free_03,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
tRNA_free_04,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
tRNA_free_05,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
tRNA_free_06,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [14]:
dfc.to_pickle(dirname + '/dfc' + descriptor + '.p')

In [15]:
dft.to_pickle(dirname + '/dft' + descriptor + '.p')

In [16]:
dfg.to_pickle(dirname + '/dfg' + descriptor + '.p')

Here we start the analysis (old notebook 17):

In [17]:
dfc['collisions_percentage'] = dfc['collisions'] / (dfc['collisions'] + dfc['nocollisions'])

In [18]:
dfc

,ribosomes,transcripts,collisions,nocollisions,peptide_bonds,collisions_percentage
2,1.0,6000.0,0.0,104.0,1090.0,0.000000
5,2.0,6000.0,0.0,205.0,2155.0,0.000000
7,3.0,6000.0,0.0,310.0,3134.0,0.000000
10,5.0,6000.0,0.0,501.0,5270.0,0.000000
13,8.0,6000.0,1.0,796.0,8425.0,0.001255
1,10.0,6000.0,3.0,990.0,10469.0,0.003021
4,20.0,6000.0,21.0,1997.0,21050.0,0.010406
6,30.0,6000.0,45.0,2935.0,31335.0,0.015101
9,50.0,6000.0,114.0,4917.0,52499.0,0.022660
12,80.0,6000.0,282.0,7956.0,84821.0,0.034232


In [19]:
ax = dfc.plot(x='ribosomes', y='collisions_percentage', figsize=(10, 7), fontsize=16)
ax.legend(["5 transcripts"], prop={'size': 16})
#plt.show()

In [20]:
dft_grouped = dft.groupby(['geneID', 'init_rate', 'length', 'ribosomes', 'transcripts'], as_index=False).mean()
dft_grouped

,geneID,init_rate,length,ribosomes,transcripts,index,tic_toc
0,0,0.100,30,1,6000,962.263736,1.525275
1,0,0.100,30,2,6000,956.242938,1.527966
2,0,0.100,30,3,6000,1000.780769,1.574615
3,0,0.100,30,5,6000,973.763547,1.581691
4,0,0.100,30,8,6000,1006.528239,1.535714
5,0,0.100,30,10,6000,1028.229195,1.574113
6,0,0.100,30,20,6000,993.030988,1.557408
7,0,0.100,30,30,6000,994.411318,1.575211
8,0,0.100,30,50,6000,991.092376,1.556521
9,0,0.100,30,80,6000,998.614674,1.543556
